In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc
import wandb 
from fastai.callback.wandb import WandbCallback
from fastxtend.vision.all import EMACallback

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    split_id = Path('../eda/fold_split.csv')
    bs = 64
    num_workers = 12
    device = 'cuda'
    seed = 2023
    out = 'exp_34'
    dataset_name = 'RNA_DatasetBaselineSplitssbppV6'
    sn_train = False
    
    model_name = 'RNA_ModelV26'
    model_kwargs = dict(dim=192 * 2,
        depth=4,
        head_size=32,
        drop_pat_dropout=0.2,
        dropout=0.2,
        bpp_transfomer_depth = 4)

   
    epoch = 64
    lr = 5e-4
    wd = 0.05
    pct_start = 0.02
    
    wandb = True
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

In [4]:
def class_to_dict(cls):
    # Create a dictionary from the class attributes
    return {key: value for key, value in cls.__dict__.items() if not key.startswith("__") and not callable(value)}

In [5]:
fns = list(CFG.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
bpp_df.drop_duplicates(subset = 'sequence_id', inplace=True)
ss = pd.read_parquet(CFG.pathss)[["sequence_id", "ss_full"]]
df = pd.read_parquet(CFG.path/'train_data.parquet')
split = pd.read_csv(CFG.split_id)
df = pd.merge(df, split, on='sequence_id')
df = pd.merge(df, bpp_df, on='sequence_id')
df = pd.merge(df, ss,  on='sequence_id')
#df = df.query("SN_filter==1").reset_index(drop=True)
df_train = df.query('is_train==True').reset_index(drop=True)
df_valid = df.query('is_train==False').reset_index(drop=True)


ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train',sn_train=CFG.sn_train)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            num_workers=CFG.num_workers, batch_size = CFG.bs, 
            persistent_workers=True, shuffle=True), CFG.device)


ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval')
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_size = CFG.bs * 2, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
del bpp_df
del ss
del split
del df
gc.collect()


16

In [6]:
if CFG.wandb:
    wandb.init(
        project="rna",
        entity="drhb",
        name=CFG.out,
        config=class_to_dict(CFG),
    )
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)(**CFG.model_kwargs).cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                     WandbCallback(log_preds=False),
                    CSVLogger(),
                     EMACallback(),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)
wandb.finish()

wandb: Currently logged in as: drhb. Use `wandb login --relogin` to force relogin


Could not gather input dimensions


epoch,train_loss,valid_loss,mae,time
0,0.125464,0.214744,0.168995,40:24
1,0.118939,0.154301,0.148176,40:32
2,0.117586,0.130829,0.139710,41:03
3,0.116861,0.128841,0.137711,41:06
4,0.116413,0.127327,0.136276,41:03
5,0.116912,0.126643,0.135629,41:09
6,0.115527,0.126343,0.135266,41:13
7,0.115672,0.125651,0.134576,41:15
8,0.115159,0.125351,0.134188,41:14
9,0.115632,0.124669,0.133565,41:15


Better model found at epoch 0 with mae value: 0.16899461993202924.
Better model found at epoch 1 with mae value: 0.14817628748720224.
Better model found at epoch 2 with mae value: 0.13971030177741292.
Better model found at epoch 3 with mae value: 0.13771089791432542.
Better model found at epoch 4 with mae value: 0.13627635192019125.
Better model found at epoch 5 with mae value: 0.13562918544605182.
Better model found at epoch 6 with mae value: 0.135266471690597.
Better model found at epoch 7 with mae value: 0.13457564457324775.
Better model found at epoch 8 with mae value: 0.13418828551837678.
Better model found at epoch 9 with mae value: 0.1335646725042834.
Better model found at epoch 10 with mae value: 0.13318182466767162.
Better model found at epoch 11 with mae value: 0.13300606534114445.
Better model found at epoch 12 with mae value: 0.13265457733859806.
Better model found at epoch 13 with mae value: 0.13240307695291842.
Better model found at epoch 14 with mae value: 0.132185296629

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:18                                                                                   │
│                                                                                                  │
│   15 │   │   │   │   │    EMACallback(),                                                         │
│   16 │   │   │   │   │   SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],             │
│   17 │   │   │   │   metrics=[MAE()]).to_fp16()                                                  │
│ ❱ 18 learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)           │
│   19 wandb.finish()                                                                              │
│   20                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/fastai/callback/schedule.py:119 in fit_one_cycle         │
│                                                                                                  │
│   116 │   lr_max = np.array([h['lr'] for h in self.opt.hypers])                                  │
│   117 │   scheds = {'lr': combined_cos(pct_start, lr_max/div, lr_max, lr_max/div_final),         │
│   118 │   │   │     'mom': combined_cos(pct_start, *(self.moms if moms is None else moms))}      │
│ ❱ 119 │   self.fit(n_epoch, cbs=ParamScheduler(scheds)+L(cbs), reset_opt=reset_opt, wd=wd, sta   │
│   120                                                                                            │
│   121 # %% ../../nbs/14_callback.schedule.ipynb 50                                               │
│   122 @patch                                                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/fastai/learner.py:264 in fit                             │
│                                                                                                  │
│   261 │   │   │   if wd is not None: self.opt.set_hypers(wd=wd)                                  │
│   262 │   │   │   self.opt.set_hypers(lr=self.lr if lr is None else lr)                          │
│   263 │   │   │   self.n_epoch = n_epoch                                                         │
│ ❱ 264 │   │   │   self._with_events(self._do_fit, 'fit', CancelFitException, self._end_cleanup   │
│   265 │                                                                                          │
│   266 │   def _end_cleanup(self): self.dl,self.xb,self.yb,self.pred,self.loss = None,(None,),(   │
│   267 │   def __enter__(self): self(_before_epoch); return self                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/fastai/learner.py:199 in _with_events                    │
│                                                                                                  │
│   196 │   │   self.xb,self.yb = b[:i],b[i:]                                                      │
│   197 │                                                                                          │
│   198 │   def _with_events(self, f, event_type, ex, final=noop):                                 │
│ ❱ 199 │   │   try: self(f'before_{event_type}');  f()                                            │
│   200 │   │   except ex: self(f'after_cancel_{event_type}')                                      │
│   201 │   │   self(f'after_{event_type}');  final()                                              │
│   202                                                                                            │
│                                                            

In [ ]:
res = dict()
for l in df_valid["L"].unique():
    ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval')
    ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval', mask_only=True)
    sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
    len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
                   drop_last=False)
    dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
                   batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

    data = DataLoaders(dl_val,dl_val)
    learn = Learner(data,
                    getattr(rnacomp.models, CFG.model_name)(dim = CFG.dim, graph_layers=CFG.graph_layers).cuda(), 
                    path = CFG.out, 
                    loss_func=loss_laplace,
                    metrics=[MAE()]).to_fp16() 
    learn.load('model')
    learn.eval()
    loss_, score_ = learn.validate()
    res[l]= score_

res = pd.DataFrame(pd.Series(res)).reset_index()
res.columns = ['L', 'mae']
res.sort_values(by="L")

In [ ]:
res

In [ ]:
res